In [18]:
import pandas as pd
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import os
import numpy as np
import schedule
import time
from datetime import date
import yagmail
import keyring


yf.pdr_override()

In [19]:
# List of tickers to iterate through. 

dow_list = ["AXP", "AMGN", 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', "GS", "HD", "HON", "IBM", "INTC", "JNJ", "KO", "JPM", "MCD",
           "MMM", "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "CRM", "VZ", "V", "WBA", "WMT", "DIS", "DOW"]


ev_list = ["AYRO", "NRG", "IDRV", "DRIV", "JCI", "RIDE", "SOLO", "LI", "VWAGY", "NKLA", "NIO", "TSLA", "TM", 'WKHS']


semiconductor_list = ["TSM", "ASML", "QCOM", "AMD", "NXPI", "SWKS", "TXN", "MRAAY", "NXPI", "IFX", "MU", "IFNNY"] 


biotech_list = ["BDSI", "ARDX", "APTO", "AGLE", "BCRX", "ASMB", "DVAX", "WVE", "KDMN", "ISEE", "SRPT", "ALNY", "INCY",
                "ILMN", "REGN", "BMRN", "AMGN", "VRTX", "ALXN", "BIIB", "GILD", "HZNP"]

gaming_list = ["PENN", "DKNG", "CZR", "BETZ", "MGM"]


flight_list = ["UAL", "DAL", "AAL", "SAVE", "LUV", "JBLU", "ALK"]


fintech_list = ["SQ", "PYPL", "GS", "GDOT", "MELI"]


blockchain_list = ["NVDA", "CME", "SQ", "IBM", "MA", "DOCU", "AMZN"]

# Random stocks to test
random_list = ["FUNC", "SPPI", "FXO", "MGPI", "GWRS", "NUV", "FLGE", "NCMI", "ODP", "CHKEL", "OZK", "FRHC"]

energy_list = ["EOG", "LNG", "BP", "EPD", "SU", "CVX", "RUN", "ENPH", "DVN"]

infrastructure_list = ['FAST', "MLM", "NSC", "UNP"]

total_list = (dow_list + ev_list + semiconductor_list + biotech_list + flight_list + fintech_list + blockchain_list +
             energy_list + infrastructure_list)

In [20]:
# Creates an empty dictionary, then grabs the data from yahoo finance

start_date = dt.datetime.now() - dt.timedelta(days=90)
end_date = dt.datetime.now()
ticker_dictionary = {}

# Specify chosen list of stocks below
for i in infrastructure_list:
    tickerSymbol = i
    tickerData = yf.Ticker(tickerSymbol)
    tickerDF = tickerData.history(period = '1d', start=start_date, end=end_date)
    ticker_dictionary[i] = tickerDF

In [21]:
percent_change = []

def stock_picks(trigger):
    purchase = 0
    Buy = []
    Sell = []
    for i in range(0, len(trigger)):
        Close = df["Close"][i]
        Close = round(Close, 2)
        
        if trigger['MACD'][i] > trigger['Trigger'][i]:
            Sell.append(np.nan)
            
            if purchase != 1:
                Buy.append(trigger['Close'][i])
                purchase = 1
                bp = Close
                print("Buying @ " + str(round(Close, 2)) + " on " + str(df.index[i]))

            else:
                Buy.append(np.nan)

        elif trigger['MACD'][i] < trigger['Trigger'][i]:
            Buy.append(np.nan)
            
            if purchase != 0:
                Sell.append(trigger['Close'][i])
                purchase = 0
                sp = Close
                pc = (sp/bp - 1) * 100
                pc = round(pc, 2)
                percent_change.append(pc)
                print("Sell now @ " +str(round(Close, 2)) + " on " + str(df.index[i]))
                
            else:
                Sell.append(np.nan)

        else:
            Sell.append(np.nan)
            Buy.append(np.nan)
         

            
    
    print(percent_change)
    print("     TOTAL GAINS/LOSSES: " + str(sum(percent_change)) + "%")

    
    return(Buy, Sell)

In [22]:
# calculate short/long term EMA, MACD, and signal line indicators

for i in ticker_dictionary:

    df = ticker_dictionary[i]

# the short and long EMAs can easily be changed by modifying the span.

# short ema 

    short = df.Close.ewm(span=8, adjust=False).mean()

# long ema

    long = df.Close.ewm(span=21, adjust=False).mean()

# MACD line

    MACD = short - long

# signal line

    Trigger_line = MACD.ewm(span=5, adjust = False).mean()

# Create new columns for data
    df["MACD"] = MACD
    df['Trigger'] = Trigger_line

    percent_change = []
    
    
    print("****CHECKING STOCK " + str(i) + "****")
    
# call stock_picks function
    test = stock_picks(df)
    

    df['Buy_Trigger'] = test[0]
    df['Sell_Trigger'] = test[1]

****CHECKING STOCK FAST****
Buying @ 49.31 on 2020-12-21 00:00:00
Sell now @ 48.79 on 2020-12-23 00:00:00
Buying @ 49.22 on 2020-12-24 00:00:00
Sell now @ 48.77 on 2020-12-30 00:00:00
Buying @ 49.41 on 2021-01-07 00:00:00
Sell now @ 48.41 on 2021-01-20 00:00:00
Buying @ 47.12 on 2021-02-04 00:00:00
Sell now @ 46.32 on 2021-02-22 00:00:00
Buying @ 46.8 on 2021-02-23 00:00:00
Sell now @ 46.28 on 2021-02-25 00:00:00
Buying @ 48.47 on 2021-03-01 00:00:00
Sell now @ 44.17 on 2021-03-04 00:00:00
Buying @ 46.37 on 2021-03-11 00:00:00
[-1.05, -0.91, -2.02, -1.7, -1.11, -8.87]
     TOTAL GAINS/LOSSES: -15.66%
****CHECKING STOCK MLM****
Buying @ 278.22 on 2020-12-28 00:00:00
Sell now @ 273.62 on 2020-12-29 00:00:00
Buying @ 280.46 on 2020-12-30 00:00:00
Sell now @ 303.07 on 2021-01-15 00:00:00
Buying @ 300.4 on 2021-02-08 00:00:00
Sell now @ 336.59 on 2021-03-02 00:00:00
[-1.65, 8.06, 12.05]
     TOTAL GAINS/LOSSES: 18.46%
****CHECKING STOCK NSC****
Buying @ 232.18 on 2020-12-23 00:00:00
Sell no

In [23]:
# Daily Buy and Sell lists

yesterday = []
sell_today = []
buy_today = []


for i in ticker_dictionary:
    new_df = ticker_dictionary[i]
    new_df["Stock"] = str(i)
    last_rows = new_df.iloc[[-1], [9, 10, 11]]
    last_rows = last_rows.dropna(axis='rows', thresh=2)
    
    buy_stock = last_rows.dropna(subset = ["Buy_Trigger"])
    buy_stock = buy_stock[["Stock", "Buy_Trigger"]]
    buy_stock = buy_stock.rename_axis("", axis = 0)
    buy_stock.columns = [''] * len(buy_stock.columns)
    
    if not buy_stock.empty:
        buy_today.append(buy_stock)
        
    sell_stock = last_rows.dropna(subset = ["Sell_Trigger"])
    sell_stock = sell_stock[["Stock", "Sell_Trigger"]]
    sell_stock = sell_stock.rename_axis("", axis = 0)
    sell_stock.columns = [''] * len(sell_stock.columns)
    
    if not sell_stock.empty:
        sell_today.append(sell_stock)
    
    if not last_rows.empty:
        yesterday.append(last_rows)

print("Buy list: " + str(buy_today))
print("Sell list: " + str(sell_today))

Buy list: []
Sell list: []


In [24]:
# Yagmail will only work if the sender_email is a Gmail email. 
# A link to futher documentation is included in the README

sender_email = os.environ.get("SENDER_EMAIL")
receiver_email = os.environ.get('RECEIVER_EMAIL')
sender_password = os.environ.get("SENDER_PASSWORD")

yag = yagmail.SMTP(user=sender_email, password=sender_password)

subject = "Daily Stock Trades"

contents = [
  '"I think you will find',
    "\n",
   'When your death takes its toll',
    "\n",
    'All the money you made',
    "\n",
    'Will never buy back your soul" - Bob Dylan',
    "\n",
    "Here is Today's Buy List: ", 
    "\n",
     str(buy_today),
    "\n",
    "Here is Today's Sell List: ",
    "\n", 
     str(sell_today),
    ]

yag.send(receiver_email, subject, contents)

{}